### Thống kê số lượng từ trong từng bộ Data

In [10]:
import pandas as pd
import time
import json
from modelling import NER
from define_name import PATH_MODEL, PATH_CONFIG, FLAG_STRICT

from dataloader import BertDataLoader

In [2]:
def loading(config, PATH):
    start = time.time()
    print("1. Loading some package")
    ner = NER(config)
    print(f"===== Done !!! =====Time: {time.time() -start:.4} s =========")
    print('2.Load model')
    start = time.time()
    ner.load_model(PATH)
    print(f"===== Done !!! =====Time: {time.time() -start:.4} s =========")
    return ner

In [3]:
with open(PATH_CONFIG, 'r', encoding= 'utf-8') as f:
    config = json.load(f)

ner = loading(config, PATH_MODEL)

1. Loading some package


Some weights of XLMRobertaModel were not initialized from the model checkpoint at pretrain/xlmr_Model and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


===== Done !!! =====Time: 5.366 s =========
2.Load model
===== Done !!! =====Time: 2.125 s =========


### Create data loader

In [4]:
dir_path = '/Users/phamvanmanh/Documents/GitHub/NER_VLSP2021/dataset/test_update_10t01.pkl'
data_loaded, _ = ner.dataloader.create_dataloader(dir = dir_path, is_train =  False)

### Predict real sententce 

In [9]:
rs = ner.predict("We present the first multi-task learning model – named PhoNLP – for joint Vietnamese part-of-speech (POS) tagging, named entity recognition (NER) and dependency parsing. Experiments on Vietnamese benchmark datasets show that PhoNLP produces state-of-the-art results, outperforming a single-task learning approach that fine-tunes the pre-trained Vietnamese language model PhoBERT (Nguyen and Nguyen, 2020) for each task independently. We publicly release PhoNLP as an open-source toolkit under the Apache License 2.0. Although we specify PhoNLP for Vietnamese, our PhoNLP training and evaluation command scripts in fact can directly work for other languages that have a pre-trained BERT-based language model and gold annotated corpora available for the three tasks of POS tagging, NER and dependency parsing. We hope that PhoNLP can serve as a strong baseline and useful toolkit for future NLP research and applications to not only Vietnamese but also the other languages. Our PhoNLP is available at https://github.com/VinAIResearch/PhoNLP")
print("Chưa sử dụng batch size")
print("Thời lượng {} dự đoán (bao gồm pre & post) 1 câu".format(rs["predict_time"]))
print("Thời lượng {} dự đoán 1 câu".format(rs["model_time"]))
print("số lượng {} sentences dự đoán".format(rs["subcut"]))

Chưa sử dụng batch size
Thời lượng 2.3040971755981445 dự đoán (bao gồm pre & post) 1 câu
Thời lượng 2.301755905151367 dự đoán 1 câu
số lượng 12 sentences dự đoán


In [ ]:
rs = ner.predict("We present the first multi-task learning model – named PhoNLP – for joint Vietnamese part-of-speech (POS) tagging, named entity recognition (NER) and dependency parsing. Experiments on Vietnamese benchmark datasets show that PhoNLP produces state-of-the-art results, outperforming a single-task learning approach that fine-tunes the pre-trained Vietnamese language model PhoBERT (Nguyen and Nguyen, 2020) for each task independently. We publicly release PhoNLP as an open-source toolkit under the Apache License 2.0. Although we specify PhoNLP for Vietnamese, our PhoNLP training and evaluation command scripts in fact can directly work for other languages that have a pre-trained BERT-based language model and gold annotated corpora available for the three tasks of POS tagging, NER and dependency parsing. We hope that PhoNLP can serve as a strong baseline and useful toolkit for future NLP research and applications to not only Vietnamese but also the other languages. Our PhoNLP is available at https://github.com/VinAIResearch/PhoNLP")
print("Chưa sử dụng batch size")
print("Thời lượng {} dự đoán (bao gồm pre & post) 1 câu".format(rs["predict_time"]))
print("Thời lượng {} dự đoán 1 câu".format(rs["model_time"]))
print("số lượng {} sentences dự đoán".format(rs["subcut"]))

### INTEFERENT TIME (SPEED)

In [6]:
import torch

avg_token_batchs = []
count = 0
epoch = 20

for bacth in data_loaded:
    b_input, b_mask, b_target = bacth
    avg_token_batch = int(torch.sum(b_mask))/data_loaded.batch_size
    avg_token_batchs.append(avg_token_batch)
    
    if count == epoch:
        break
    count += 1

rs_evalue = ner.evaluate(data_loaded, epoch= epoch)

In [7]:
print("lượng token trung bình:{}".format(sum(avg_token_batchs)/len(avg_token_batchs)))
print("dự đoán {} sentences trong 1s sử dụng CPU".format(data_loaded.batch_size*epoch/rs_evalue["batch_time"]))

lượng token trung bình:23.523809523809526
dự đoán 7.042997614334391 sentences trong 1s sử dụng CPU


### ACCURACY

In [8]:
epoch = 10
rs_evalue = ner.evaluate(data_loaded, epoch= epoch, strict = FLAG_STRICT["MIN"])
print("accuracy by word f1-score: {}".format(rs_evalue["f1"]))
rs_evalue = ner.evaluate(data_loaded, epoch= epoch, strict = FLAG_STRICT["MAX"])
print("accuracy by group word f1-score: {}".format(rs_evalue["f1"]))

accuracy by word f1-score: 0.9210098528470911
accuracy by group word f1-score: 0.7359104155442828


### Nhận xét:
- time inference phụ thuộc vào max len
- sủ dụng bacth size  phụ thuộc vào max len

In [11]:
from dataloader_real import BertDataLoaderReal
data = "We present the first multi-task learning model – named PhoNLP – for joint Vietnamese part-of-speech (POS) tagging, named entity recognition (NER) and dependency parsing. Experiments on Vietnamese benchmark datasets show that PhoNLP produces state-of-the-art results, outperforming a single-task learning approach that fine-tunes the pre-trained Vietnamese language model PhoBERT (Nguyen and Nguyen, 2020) for each task independently. We publicly release PhoNLP as an open-source toolkit under the Apache License 2.0. Although we specify PhoNLP for Vietnamese, our PhoNLP training and evaluation command scripts in fact can directly work for other languages that have a pre-trained BERT-based language model and gold annotated corpora available for the three tasks of POS tagging, NER and dependency parsing. We hope that PhoNLP can serve as a strong baseline and useful toolkit for future NLP research and applications to not only Vietnamese but also the other languages. Our PhoNLP is available at https://github.com/VinAIResearch/PhoNLP"
data = data.split()

data_loaded = ner.dataloaderreal.create_dataloader(data, is_train =  False)
